In [4]:
from langchain.memory import  ConversationBufferMemory


In [6]:
memory = ConversationBufferMemory(
	return_messages=True
)

memory.save_context({"input": "Hello!"}, {"output": "How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hello!'),
  AIMessage(content='How are you?')]}